In [1]:
import dpeeg
from dpeeg import transforms
import numpy as np

print(dpeeg.__version__)

0.4.0


In [2]:
dpeeg.EEGData()

[edata=(0,), label=(0,)]

In [4]:
dpeeg.EEGData(
    edata=np.random.rand(16, 3, 20),
    label=np.random.randint(0, 3, 16),
    adj=np.random.rand(3, 3)
)

[edata=(16, 3, 20), label=(16,), adj=(3, 3)]

In [2]:
eegdata = dpeeg.EEGData(np.random.randn(16, 3, 10), np.random.randint(0, 3, 16))
eegdata.shape

(3, 10)

In [3]:
eegdata = dpeeg.EEGData(np.random.randn(16, 3, 10), np.random.randint(0, 3, 16))
transforms.SplitTrainTest(0.2)(eegdata, verbose=False)

Train: [edata=(12, 3, 10), label=(12,)]
Test : [edata=(4, 3, 10), label=(4,)]

In [4]:
multi_sess_eegdata = dpeeg.MultiSessEEGData([eegdata.copy() for _ in range(4)])
transforms.SplitTrainTest(
    cross=True, 
    train_sessions=[f"session_{i + 1}" for i in range(2)], 
    test_sessions=[f"session_4"]
)(multi_sess_eegdata, verbose=False)

Train: [edata=(32, 3, 10), label=(32,)]
Test : [edata=(16, 3, 10), label=(16,)]

In [5]:
transforms.SplitTrainTest(0.5)(multi_sess_eegdata, verbose=False)

Train: [edata=(32, 3, 10), label=(32,)]
Test : [edata=(32, 3, 10), label=(32,)]

In [6]:
transforms.MinMaxNorm()(eegdata)

  Apply MinMaxNorm(min=None, max=None, dim=None) ...


[edata=(16, 3, 10), label=(16,)]

In [7]:
transforms.ZscoreNorm()(eegdata)

  Apply ZscoreNorm(mean=None, std=None, dim=None) ...


[edata=(16, 3, 10), label=(16,)]

In [8]:
eegdata = dpeeg.EEGData(np.random.randn(16, 3, 10), np.random.randint(0, 3, 16))
transforms.Crop(4, 9)(eegdata, verbose=False)

[edata=(16, 3, 5), label=(16,)]

In [9]:
eegdata = dpeeg.EEGData(np.random.randn(16, 3, 10), np.random.randint(0, 3, 16))
transforms.Unsqueeze(dim=2)(eegdata, verbose=False)

[edata=(16, 3, 1, 10), label=(16,)]

In [10]:
eegdata = dpeeg.EEGData(np.random.randn(16, 3, 10), np.random.randint(0, 3, 16))
transforms.FilterBank(250)(eegdata, verbose=False)

[edata=(16, 9, 3, 10), label=(16,)]

In [11]:
eegdata = dpeeg.EEGData(np.random.randn(16, 3, 10), np.random.randint(0, 3, 16))
def expand_dim(data, dim=1):
    data["edata"] = np.expand_dims(data["edata"], dim)
transforms.ApplyFunc(expand_dim, dim=0)(eegdata, verbose=False)

[edata=(1, 16, 3, 10), label=(16,)]

In [12]:
split_eegdata = dpeeg.SplitEEGData(eegdata, eegdata.copy())
transforms.ApplyFunc(expand_dim, ["train"])(split_eegdata, verbose=False)

Train: [edata=(1, 1, 16, 3, 10), label=(16,)]
Test : [edata=(1, 16, 3, 10), label=(16,)]

In [13]:
eegdata = dpeeg.EEGData(np.random.randn(16, 1, 3, 10), np.random.randint(0, 3, 16))
transforms.Squeeze()(eegdata, verbose=False)

[edata=(16, 3, 10), label=(16,)]

In [14]:
eegdata = dpeeg.EEGData(np.random.randn(16, 3, 10), np.random.randint(0, 3, 16))
eegdata['label']

array([0, 1, 2, 0, 1, 2, 1, 1, 2, 1, 0, 2, 0, 2, 0, 0])

In [15]:
transforms.PickLabel(np.array([1, 2]))(eegdata, verbose=False)
eegdata['label']

array([1, 0, 1, 0, 1, 0, 1, 0, 0, 1])

In [16]:
eegdata = dpeeg.EEGData(
    edata=np.random.randn(16, 3, 10),
    label=np.random.randint(0, 3, 16),
    adj=np.random.randn(16, 3, 3),
    pcc=np.random.randn(16, 3, 3),
)
transforms.PickLabel(
    np.array([0, 1]), keys=["edata", "adj"]
)(eegdata, verbose=False)

[edata=(12, 3, 10), label=(12,), adj=(12, 3, 3), pcc=(16, 3, 3)]

In [17]:
eegdata = dpeeg.EEGData(np.random.randn(16, 3, 10), np.random.randint(2, 5, 16))
eegdata['label']

array([2, 3, 3, 3, 3, 3, 4, 2, 3, 2, 2, 3, 4, 3, 4, 4])

In [18]:
transforms.LabelMapping(np.array([[2, 3, 4], [0, 0, 1]]))(eegdata, verbose=False)
eegdata["label"]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1])

In [19]:
eegdata = dpeeg.EEGData(np.random.randn(16, 3, 10), np.random.randint(0, 3, 16))
split_eegdata = dpeeg.SplitEEGData(eegdata.copy(), eegdata.copy())
transforms.SegRecTime(2, 3)(split_eegdata, verbose=False)

Train: [edata=(64, 3, 10), label=(64,)]
Test : [edata=(16, 3, 10), label=(16,)]

In [20]:
eegdata = dpeeg.EEGData(np.random.randn(16, 3, 10), np.random.randint(0, 3, 16))
split_eegdata = dpeeg.SplitEEGData(eegdata.copy(), eegdata.copy())
transforms.SlideWinAug(2)(split_eegdata, verbose=False)

Train: [edata=(80, 3, 2), label=(80,)]
Test : [edata=(16, 3, 2), label=(16,)]

In [21]:
eegdata = dpeeg.EEGData(np.random.randn(16, 3, 10), np.random.randint(0, 3, 16))
transforms.SlideWin(3, 1)(eegdata, verbose=False)

[edata=(64, 3, 3), label=(64,)]

In [22]:
eegdata = dpeeg.EEGData(np.random.randn(16, 3, 10), np.random.randint(0, 3, 16))
transforms.Sequential(
    transforms.Unsqueeze(),
    transforms.Crop(2, 5),
)(eegdata, verbose=False)

[edata=(16, 1, 3, 3), label=(16,)]

In [23]:
eegdata = dpeeg.EEGData(np.random.randn(16, 3, 10), np.random.randint(0, 3, 16))
transforms.Unsqueeze()(eegdata, verbose=False)

[edata=(16, 1, 3, 10), label=(16,)]

In [24]:
eegdataset = dpeeg.datasets.EEGDataset([
    eegdata.copy(), eegdata.copy(), eegdata.copy()
])
transforms.Squeeze()(eegdataset, iter=False, verbose=False)

[EEGDataset:
  [eegdataset]: Subjects=3, type=EEGData
  [event_id]: None
]

In [25]:
tran = transforms.Unsqueeze()
for subject, eegdata in tran(eegdataset, iter=True, verbose=False):
    print(subject, eegdata)

0 [edata=(16, 1, 3, 10), label=(16,)]
1 [edata=(16, 1, 3, 10), label=(16,)]
2 [edata=(16, 1, 3, 10), label=(16,)]


In [26]:
eegdata = dpeeg.EEGData(np.random.randn(16, 3, 10), np.random.randint(0, 3, 16))
dpeeg.MultiSessEEGData([eegdata.copy(), eegdata.copy()])

{'session_1': [edata=(16, 3, 10), label=(16,)],
 'session_2': [edata=(16, 3, 10), label=(16,)]}

In [27]:
dpeeg.MultiSessEEGData({'xx': eegdata.copy(), 'yy': eegdata.copy()})

{'xx': [edata=(16, 3, 10), label=(16,)],
 'yy': [edata=(16, 3, 10), label=(16,)]}

In [28]:
split_eegdata = dpeeg.SplitEEGData(eegdata.copy(), eegdata.copy())

In [29]:
eegdata = dpeeg.EEGData(np.random.randn(16, 3, 10), np.random.randint(2, 5, 16))
eegdata.index([1, 2, 3])

[edata=(3, 3, 10), label=(3,)]

In [30]:
eegdata.index(slice(6))

[edata=(6, 3, 10), label=(6,)]

In [32]:
eegdata = dpeeg.EEGData(np.random.randn(16, 3, 10), np.random.randint(0, 3, 16))
split_eegdata = dpeeg.SplitEEGData(eegdata, eegdata)
split_eegdata

Train: [edata=(16, 3, 10), label=(16,)]
Test : [edata=(16, 3, 10), label=(16,)]